In [1]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [2]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


In [3]:
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [4]:
atr=bytes(connection.getATR())
atr.hex()

'3bdc18ff8191fe1fc38073c821136605036351000250'

In [5]:
def select(appletID):
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print(bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [10]:
select("B00B5111CA01")

00A4040006B00B5111CA0100


b''

In [11]:
def request(APDU):
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [12]:
# get secret
request("B0A1000000")

b'I am a teapot gimme some tea plz'

In [13]:
secret = "kitchen stick vault rain coast nephew copy afford monitor decade chalk segment capable forward cliff thank pave thunder diamond behave draft dial frozen essence"
print(len(secret))
data = bytes([len(secret)])+secret.encode()
data.hex()

160


'a06b69746368656e20737469636b207661756c74207261696e20636f617374206e657068657720636f7079206166666f7264206d6f6e69746f7220646563616465206368616c6b207365676d656e742063617061626c6520666f727761726420636c696666207468616e6b2070617665207468756e646572206469616d6f6e6420626568617665206472616674206469616c2066726f7a656e20657373656e6365'

In [14]:
# put secret
request("B0A20000"+data.hex())

b'kitchen stick vault rain coast nephew copy afford monitor decade chalk segment capable forward cliff thank pave thunder diamond behave draft dial frozen essence'

In [15]:
# get secret again
request("B0A10000")

b'kitchen stick vault rain coast nephew copy afford monitor decade chalk segment capable forward cliff thank pave thunder diamond behave draft dial frozen essence'

In [16]:
connection.disconnect()